# Practica 4
## Captcha Destroyer :por: Python Eaters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

## Import


In [1]:
import string

from tensorflow.keras import layers
import os
import numpy as np 
import cv2 #OpenCV

ModuleNotFoundError: No module named 'tensorflow.python'

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos cv2 (OpenCV)

Primero necesitamos saber cuales son los caracteres que nos podemos encontrar en los 'catcha' y, ya que estamos, la cantidad de caracteres que son.

In [ ]:
symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
numChar = len(symbols) # numero de chars posibles
numChar

Luego de cuantas imagenes es unuestro Dataset

In [ ]:
#tamanio del dataset 
directory_examples = "C:\Users\NITROPC\OneDrive\Escritorio\Daniel\Trabajo\Universidad\4º Año\2ºCuatri\SI\Practica 4\SI_CaptchaDestroller\samples"
n = len(os.listdir(directory_examples))
n

Despues queremos preprocesar las imagenes con las que estemos trabajando, para asegurarnos que todas tengan el mismo formato y dimensiones.
Notese que el nombre de la imagen es la solucion del catchap.

In [ ]:
def preprocesar_img():
    img_array = np.zeros((n,50,200,1)) # n = 1070 (tamanio del dataset ) * 50 (altura de la imagen) * 200 (anchura de la imagen) (con todas las entradas a 0)
    letersImg_array = np.zeros((5,n,numChar)) # 5 (letras de catchap) * n = 1070 (tamanio del dataset ) * 36 (posibles caracteres)  (con todas las entradas a 0)

    for i, pic in enumerate(os.listdir(directory_examples)):
    # i : indice en el que se encuentra la imagen en el directorio (0-(n-1))->(n)
    # pic : contiene el nombre de la imagen en posicion i, que es equivalente a la solucion del catchap + .png
        
        img = cv2.imread(os.path.join(directory_examples, pic), cv2.IMREAD_GRAYSCALE) #guardamos la imagen en forma de matriz en una escala de blanco y negro
        pic_target = pic[:-4]# guardamos el objetimo (solucion del catchap), que es quivalente a el nombre - .png (.png = 4bits => -4)

        if len(pic_target) < 6: #solo captcha de 5 letras o menos
            img = img / 255.0 #escalar la imagen entre 0 y 1
            img = np.reshape(img, (50, 200, 1)) #redimensionas el array a 200 de ancho, 50 de alto , en channel 1 

            target=np.zeros((5,numChar)) #5 (letras de catchap) * 36 (posibles caracteres)  (con todas las entradas a 0) = (ej: 0:0,0,....(36),0; 0:0,0,...)

            for j, k in enumerate(pic_target):
            # j : indice en el que se encuentra la palabra (0-4)->(5)
            # k : letra correspondiente a la pos j
                indexSymbol = symbols.find(k) # indexSymbol guarda la pos de la letra k, respecto al array de  symbols
                target[j, indexSymbol] = 1 # en la posicion j : remplazo en la pos index el 0 por un 1 (ej: en la pos j esta la letra m)

            img_array[i] = img # guardo las imagen en pos i 
            letersImg_array[:,i] = target # guardo el target (solucion) en pos i

    return img_array, letersImg_array

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

## Parte 2: Reconocer una cadena de dígitos y letras bien separadas
Ahora que sabemos identificar letras y números por separado podemos agruparlos bien separados en una cadena para leerlos y escribirlos en orden de aparición. (logic RNN)

## Parte 3: Reconocer dígitos y letras que se juntan/mezclan
Por último queda ser capaces de distinguir los números y letras que están solapados en la cadena para poder escribirlos (CRNN = CNN + RNN)